## Análise explanatória Empresas Ativas (EMPRESAS)

#### Realização de uma análise explanatória detalhada dos dados das empresas ativas da cidade do Recife, verificando possíveis inconsistencias ou duplicações nos registros.

##### 🟩🟩 Importando Bibliotecas

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
# Diretório onde os arquivos CSV estão localizados
CSV_FOLDER = r'./datasets/CSV_Separados'

# Nome do arquivo CSV que você quer carregar
CSV_FILE = 'EMPRESAS.csv'  # Substitua pelo nome do arquivo gerado

# Caminho completo do arquivo CSV
csv_path = os.path.join(CSV_FOLDER, CSV_FILE)

# Verificar se o arquivo existe antes de tentar carregá-lo
if os.path.exists(csv_path):
    # Carregar o arquivo CSV como DataFrame
    df = pd.read_csv(csv_path)
    print(f"Arquivo CSV '{CSV_FILE}' carregado com sucesso!")
else:
    print(f"Erro: O arquivo '{csv_path}' não foi encontrado.")

pd.set_option('display.max_columns', None)  # Exibe todas as colunas
pd.set_option('display.max_colwidth', None)  # Exibe o conteúdo completo de cada célula
pd.set_option('display.width', None)  # Ajuste de largura para evitar quebra de linha

Arquivo CSV 'EMPRESAS.csv' carregado com sucesso!


In [4]:
df.head(5)

,CNPJ BASICO,RAZAO SOCIAL / NOME EMPRESARIAL,NATUREZA JURIDICA,QUALIFICACAO DO SOCIO,CAPITAL SOCIAL DA EMPRESA,PORTE DA EMPRESA,ENTE FEDERATIVO RESPONSAVEL
0,0,BANCO DO BRASIL SA,SOCIEDADE DE ECONOMIA MISTA,DIRETOR,"120000000000,00",DEMAIS,NaN
1,633,INSTITUTO QUALIDADE NO ENSINO,ASSOCIAÇÃO PRIVADA,PRESIDENTE,"0,00",DEMAIS,NaN
2,802,HC PNEUS S/A,SOCIEDADE ANÔNIMA FECHADA,DIRETOR,"204418998,00",DEMAIS,NaN
3,2927,TRANSMETA TRANSPORTES RODOVIARIOS DE CARGAS LTDA,SOCIEDADE EMPRESÁRIA LIMITADA,SÓCIO-ADMINISTRADOR,"800000,00",DEMAIS,NaN
4,5050,INTEGRE DO BRASIL S/A INDUSTRIA COM IMP EXP TRANSPORTES,SOCIEDADE ANÔNIMA FECHADA,PRESIDENTE,"0,00",DEMAIS,NaN


##### 🟡 ENTE FEDERATIVO RESPONSÁVEL tem nulls ...vamos olhar melhor

In [5]:
valores_unicos = df['ENTE FEDERATIVO RESPONSAVEL'].unique()
print(valores_unicos)

[nan 'UNIÃO' 'PERNAMBUCO' 'RECIFE - PE' 'PIAUI' 'MARAIAL - PE'
 'RIO GRANDE DO SUL' 'ALAGOAS']


##### 🟩🟩 Verificando a quantidade de nulos por coluna (mostrando apenas as colunas com nulos)


In [26]:
# Contar os valores nulos por coluna
nulos_por_coluna = df.isna().sum()

# Filtrar as colunas com valores nulos (maior que 0)
colunas_com_nulos = nulos_por_coluna[nulos_por_coluna > 0]

# Exibir as colunas com valores nulos
print(colunas_com_nulos)



QUALIFICACAO DO SOCIO              33
ENTE FEDERATIVO RESPONSAVEL    532238
dtype: int64


##### 🟩🟩 Verificando dados duplicados 

In [29]:
duplicatas = df.duplicated()

# Exibir as linhas duplicadas (se houver)
print(df[duplicatas])

Empty DataFrame
Columns: [CNPJ BASICO, RAZAO SOCIAL / NOME EMPRESARIAL, NATUREZA JURIDICA, QUALIFICACAO DO SOCIO, CAPITAL SOCIAL DA EMPRESA, PORTE DA EMPRESA, ENTE FEDERATIVO RESPONSAVEL]
Index: []


##### Não há dados duplicados na tabela empresas


##### 🟩🟩 Verificando os tipos dos dados 

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532580 entries, 0 to 532579
Data columns (total 7 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   CNPJ BASICO                      532580 non-null  int64 
 1   RAZAO SOCIAL / NOME EMPRESARIAL  532580 non-null  object
 2   NATUREZA JURIDICA                532580 non-null  object
 3   QUALIFICACAO DO SOCIO            532547 non-null  object
 4   CAPITAL SOCIAL DA EMPRESA        532580 non-null  object
 5   PORTE DA EMPRESA                 532580 non-null  object
 6   ENTE FEDERATIVO RESPONSAVEL      342 non-null     object
dtypes: int64(1), object(6)
memory usage: 28.4+ MB


##### 🟡 Em uma analise por data type é preciso ajustar 'CNPJ BASICO' para object pois ele não possui a função de soma visto que se trata de códigos

##### 🟩🟩 Alterando Dtype para object 

In [35]:
conversao = ['CNPJ BASICO']

# Converter as colunas específicas de 'int' para 'object'
df[conversao] = df[conversao].astype('object')

# Verificar os tipos de dados após a conversão
df.dtypes

CNPJ BASICO                        object
RAZAO SOCIAL / NOME EMPRESARIAL    object
NATUREZA JURIDICA                  object
QUALIFICACAO DO SOCIO              object
CAPITAL SOCIAL DA EMPRESA          object
PORTE DA EMPRESA                   object
ENTE FEDERATIVO RESPONSAVEL        object
dtype: object

##### 🟩🟩 Salvando o dataframe EMPRESAS

In [41]:
# Pasta para salvar o arquivo
OUTPUT_FOLDER = r'C:\Users\carli\Desktop\EmpresasAtivas\datasets\CSV_Tratados'
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [39]:
# Salva o DataFrame em formato CSV com separador e controle de casas decimais
df.to_csv('C:\\Users\\carli\\Desktop\\EmpresasAtivas\\datasets\\CSV_Tratados\\empresas.csv', sep=';', decimal=',', encoding='UTF-8')
print('Salvo com sucesso')